In [ ]:
import pandas as pd

data = pd.read_excel('../data/get_around_delay_analysis.xlsx')
for index, i in enumerate(data.columns):
    print(i, data.dtypes[index])
print()
print(data[data['delay_at_checkout_in_minutes'].notna()].head())

rental_id int64
car_id int64
checkin_type object
state object
delay_at_checkout_in_minutes float64
previous_ended_rental_id float64
time_delta_with_previous_rental_in_minutes float64

   rental_id  car_id checkin_type  state  delay_at_checkout_in_minutes  \
1     507750  269550       mobile  ended                         -81.0   
2     508131  359049      connect  ended                          70.0   
5     511626  398802       mobile  ended                        -203.0   
6     511639  370585      connect  ended                         -15.0   
7     512303  371242       mobile  ended                         -44.0   

   previous_ended_rental_id  time_delta_with_previous_rental_in_minutes  
1                       NaN                                         NaN  
2                       NaN                                         NaN  
5                       NaN                                         NaN  
6                  563782.0                                       570.0  
7

C:\Users\Martin\AppData\Local\Temp\ipykernel_33292\4111968636.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i, data.dtypes[index])
C:\Users\Martin\AppData\Local\Temp\ipykernel_33292\4111968636.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i, data.dtypes[index])
C:\Users\Martin\AppData\Local\Temp\ipykernel_33292\4111968636.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i, data.dtypes[index])
C:\User